Test 1 here

In [1]:
import random
from classes.minion import Minion
from classes.hero import Hero
from scenarios.deck_1 import deck  # Example deck import
from solver.slv import run_single_turn
from state.transition_state import start_turn, apply_results, end_turn


In [2]:
# 1) Create board states
friendly_minions = [
    Minion(
        name="Test Friendly Minion",
        minion_class="Neutral",
        keywords=[],
        attack=4,
        health=3,
        strat_value=1,
        mana_cost=4
    )
]

enemy_minions = [
    Minion(
        name="Test Enemy Minion",
        minion_class="Neutral",
        keywords=[],
        attack=1,
        health=4,
        strat_value=1,
        mana_cost=2
    )
]

# 2) Hand is empty at the start
hand_list = []

In [3]:
def main():
    # 1) Create Hero objects (assuming a Hero class exists)
    hero1 = Hero("Paladin", 30)
    hero2 = Hero("Warrior", 30)

    # 2) Print the deck size before drawing
    print(f"Deck has {len(deck)} cards at the start.")

    # 3) Start turn: hero1 draws exactly one card from the deck
    start_turn(hero1, deck, hand_list)

    # Check what got drawn
    print(f"After draw, hand_list has {len(hand_list)} cards.")
    for k, card in enumerate(hand_list):
        print(f"  Hand card {k}: {card.name} "
              f"(Atk={card.attack}, HP={card.health}, Cost={card.mana_cost})")

    # 4) Build arrays for the solver
    m = len(friendly_minions)   # number of friendly minions on board
    n = len(enemy_minions)      # number of enemy minions on board
    h = len(hand_list)          # number of cards in hand
    M = 5                       # Mana for this turn
    H_hero = 4                  # Opponent hero health

    # ---- Print the counts and minion details ----
    print(f"Number of friendly minions on board: {m}")
    for i, fmn in enumerate(friendly_minions):
        print(f"  - Friendly Minion {i}: {fmn.name} "
              f"(Atk={fmn.attack}, HP={fmn.health})")

    print(f"Number of enemy minions on board: {n}")
    for j, emn in enumerate(enemy_minions):
        print(f"  - Enemy Minion {j}: {emn.name} "
              f"(Atk={emn.attack}, HP={emn.health})")

    print(f"Number of cards in hand: {h}")
    for k, card in enumerate(hand_list):
        print(f"  - Hand Card {k}: {card.name} "
              f"(Atk={card.attack}, HP={card.health}, Cost={card.mana_cost})")

    # Combine board minions + hand minions for indexing in the solver
    combined = friendly_minions + hand_list
    A = [mn.attack for mn in combined]   # attack values
    B = [mn.health for mn in combined]   # health values
    P = [enm.attack for enm in enemy_minions]
    Q = [enm.health for enm in enemy_minions]

    C = [mn.mana_cost for mn in hand_list]
    S = [mn.strat_value for mn in hand_list]

    # 5) Define or load your weights
    weights = {
        "W1": 10,  # e.g. hero alive
        "W2": 9,   # board clear
        "W3": 8,   # minion->hero damage
        "W4": 7,   # penalty for leaving strong enemy minion
        "W5": 6,   # keep friendly minion health
        "W6": 5,   # minion->minion attack
        "W7": 4,   # penalty for taking damage from enemy minions
        "W8": 3    # strategic card plays
    }

    # 6) Solve (call the new run_single_turn with a weights dict)
    solution = run_single_turn(
        m=m, n=n, h=h,
        M=M, H_hero=H_hero,
        A=A, B=B, P=P, Q=Q, C=C, S=S,
        weights=weights
    )

    # 7) Apply results to scenario_data
    scenario_data = {
        "A": A,
        "B": B,
        "P": P,
        "Q": Q,
        "H_hero": H_hero,
    }
    apply_results(solution, scenario_data)

    # 8) End turn
    end_turn(hero1, hero2)
    print("Turn complete. Inspect logs above for solver outcome.")


if __name__ == "__main__":
    main()


Deck has 5 cards at the start.
Paladin draws: Minion: Wolfrider | Class: Neutral | Attack: 3 | Health: 1 | Keywords: Charge
After draw, hand_list has 1 cards.
  Hand card 0: Wolfrider (Atk=3, HP=1, Cost=3)
Number of friendly minions on board: 1
  - Friendly Minion 0: Test Friendly Minion (Atk=4, HP=3)
Number of enemy minions on board: 1
  - Enemy Minion 0: Test Enemy Minion (Atk=1, HP=4)
Number of cards in hand: 1
  - Hand Card 0: Wolfrider (Atk=3, HP=1, Cost=3)
Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: Intel(R) Core(TM) Ultra 7 165U, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: Intel(R) Core(TM) Ultra 7 165U, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 14 logical processors, using up to